In [1]:
import os
import random
from tqdm.notebook import tqdm
import pandas as pd
import pickle
import numpy as np
from glob import glob
from collections import Counter
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel, CoherenceModel
import scipy
import statsmodels.formula.api as smf
import statsmodels.api as sm
import itertools
import re
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, f1_score, classification_report
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import seaborn as sns
#
import networkx as nx
import community
#
from lda.lda_prop5 import MultiLDA
from lda.lda_prop4 import Lda, MyCorpus
from lda.lda_prop4 import MyCorpora
from lda.related_tools import *

In [2]:
# Read Multiomics data
df_comb = pd.read_pickle('data/20211013_data_SeRfSnpsBmHtDtMbPtBmDisGeNet.pkl')

df_comb.element_name.unique()

array(['Biomaker', 'RiskFactor', 'SideEffect', 'SNPs',
       'HumanTranscriptome', 'DrugTarget', 'Microbiota', 'Wikipathway',
       'BiomarkerChemical', 'BiomarkerKaryotype', 'BiomarkerGenetic',
       'BiomarkerProtein', 'DisGeNet_AlteredExpression',
       'DisGeNet_Biomarker', 'DisGeNet_CausalMutation',
       'DisGeNet_ChromosomalRearrangement', 'DisGeNet_FusionGene',
       'DisGeNet_GeneticVariation', 'DisGeNet_GenomicAlterations',
       'DisGeNet_GermlineCausalMutation',
       'DisGeNet_GermlineModifyingMutation', 'DisGeNet_ModifyingMutation',
       'DisGeNet_PosttranslationalModification',
       'DisGeNet_SomaticCausalMutation',
       'DisGeNet_SusceptibilityMutation', 'DisGeNet_Therapeutic'],
      dtype=object)

In [3]:
# Extract common 6955 diseases in three-multiomics dataset
# - DisGeNet_AlteredExpression
# - DisGeNet_Biomarker
# - DisGeNet_GeneticVariation

df_ae = df_comb.query("element_name == 'DisGeNet_AlteredExpression'")
df_bm = df_comb.query("element_name == 'DisGeNet_Biomarker'")
df_gv = df_comb.query("element_name == 'DisGeNet_GeneticVariation'")

common_disease = list(set(df_ae.key_c) & set(df_bm.key_c) & set(df_gv.key_c))

df_ae = df_ae.query("key_c in @common_disease").reset_index(drop=True)
df_bm = df_bm.query("key_c in @common_disease").reset_index(drop=True)
df_gv = df_gv.query("key_c in @common_disease").reset_index(drop=True)

display(df_ae)
display(df_bm)
display(df_gv)


,key_c,element_name,Disease,element,N
0,(idiopathic) normal pressure hydrocephalus,DisGeNet_AlteredExpression,[(idiopathic) normal pressure hydrocephalus],[TNR],1
1,11-beta-hydroxylase deficiency,DisGeNet_AlteredExpression,[11-beta-hydroxylase deficiency],"[CYP11B1, CYP2B6]",2
2,"17-alpha-hydroxylase/17,20 lyase deficiency",DisGeNet_AlteredExpression,"[17-alpha-hydroxylase/17,20 lyase deficiency]",[POMC],1
3,2-methyl-3-hydroxybutyric aciduria,DisGeNet_AlteredExpression,[2-methyl-3-hydroxybutyric aciduria],"[HSD17B10, MX1]",2
4,21-hydroxylase deficiency,DisGeNet_AlteredExpression,[21-hydroxylase deficiency],"[POMC, CRH, CYP2B6, AR, CSF2, DKK1, CYP2C19, R...",9
...,...,...,...,...,...
6950,zap70 deficiency,DisGeNet_AlteredExpression,[zap70 deficiency],[ZAP70],1
6951,zika virus infection,DisGeNet_AlteredExpression,[zika virus infection],"[STAT2, ALB, HMOX1, SOAT1, SOD2, MSI1, IFNB1, ...",51
6952,zollinger-ellison syndrome,DisGeNet_AlteredExpression,[zollinger-ellison syndrome],"[MET, IGF1R, GAST, SCTR, PPY]",5
6953,zoonoses,DisGeNet_AlteredExpression,[zoonoses],"[IFITM3, IFIT2]",2


,key_c,element_name,Disease,element,N
0,(idiopathic) normal pressure hydrocephalus,DisGeNet_Biomarker,[(idiopathic) normal pressure hydrocephalus],"[APOE, DMD, AQP4, CEBPZ, ASRGL1, CSF2, LAMC2, ...",13
1,11-beta-hydroxylase deficiency,DisGeNet_Biomarker,[11-beta-hydroxylase deficiency],"[LOC110673971, POMC, CYP11B1]",3
2,"17-alpha-hydroxylase/17,20 lyase deficiency",DisGeNet_Biomarker,"[17-alpha-hydroxylase/17,20 lyase deficiency]","[CYP17A1, POMC]",2
3,2-methyl-3-hydroxybutyric aciduria,DisGeNet_Biomarker,[2-methyl-3-hydroxybutyric aciduria],"[HSD17B10, DHCR7, ACAD8, ACADSB, AUH]",5
4,21-hydroxylase deficiency,DisGeNet_Biomarker,[21-hydroxylase deficiency],"[CYP21A2, POMC, GH1, CYP21A1P, TNXA, TNXB, IDS...",23
...,...,...,...,...,...
6950,zap70 deficiency,DisGeNet_Biomarker,[zap70 deficiency],"[CYBB, NCF1, ZAP70, PNP]",4
6951,zika virus infection,DisGeNet_Biomarker,[zika virus infection],"[IFNL1, ERVK-20, ERVW-1, ERVK-6, AXL, ALB, GAB...",153
6952,zollinger-ellison syndrome,DisGeNet_Biomarker,[zollinger-ellison syndrome],"[IL6, MEN1, GAST, CCKBR, SCT, CHGA, SST, CDKN1...",14
6953,zoonoses,DisGeNet_Biomarker,[zoonoses],"[NCKIPSD, AHI1, MMP12, CCR5, CASP12, SLC9A6, S...",14


,key_c,element_name,Disease,element,N
0,(idiopathic) normal pressure hydrocephalus,DisGeNet_GeneticVariation,[(idiopathic) normal pressure hydrocephalus],"[CSF2, LAMC2]",2
1,11-beta-hydroxylase deficiency,DisGeNet_GeneticVariation,[11-beta-hydroxylase deficiency],"[CYP11B1, CYP11B2, PPIG, CYP21A2, REN, TXNRD1,...",7
2,"17-alpha-hydroxylase/17,20 lyase deficiency",DisGeNet_GeneticVariation,"[17-alpha-hydroxylase/17,20 lyase deficiency]","[CYP17A1, CYP4F3, SRY]",3
3,2-methyl-3-hydroxybutyric aciduria,DisGeNet_GeneticVariation,[2-methyl-3-hydroxybutyric aciduria],"[HSD17B10, FSIP1, ROGDI]",3
4,21-hydroxylase deficiency,DisGeNet_GeneticVariation,[21-hydroxylase deficiency],"[CYP21A2, AZF1, CYP21A1P, POMC, TNXB, NR3C1, C...",38
...,...,...,...,...,...
6950,zap70 deficiency,DisGeNet_GeneticVariation,[zap70 deficiency],[ZAP70],1
6951,zika virus infection,DisGeNet_GeneticVariation,[zika virus infection],"[RAF1, MIR34A, IVNS1ABP, PTPN11, ERVK-32, ERVK-6]",6
6952,zollinger-ellison syndrome,DisGeNet_GeneticVariation,[zollinger-ellison syndrome],"[CXCL8, TNF, GAST]",3
6953,zoonoses,DisGeNet_GeneticVariation,[zoonoses],"[PRNP, BST2, SAMD9]",3


In [4]:
# convert to corpus for multimodal topic moddeling
# Disease - AlteredExpression
corpus_disease_ae = MyCorpus.read_corpus_from_dataframe(
    df_ae, 
    columns_title='key_c',
    columns_text='element'
)
display(corpus_disease_ae.df_chi())

# Disease - Biomarker
corpus_disease_bm = MyCorpus.read_corpus_from_dataframe(
    df_bm, 
    columns_title='key_c',
    columns_text='element'
)
display(corpus_disease_bm.df_chi())

# Disease - GeneticVariation
corpus_disease_gv = MyCorpus.read_corpus_from_dataframe(
    df_gv, 
    columns_title='key_c',
    columns_text='element'
)
display(corpus_disease_gv.df_chi())

,document_no,document,vocabulary_no,vocabulary,count
0,0,(idiopathic) normal pressure hydrocephalus,0,TNR,1
1,1,11-beta-hydroxylase deficiency,1,CYP11B1,1
2,1,11-beta-hydroxylase deficiency,2,CYP2B6,1
3,2,"17-alpha-hydroxylase/17,20 lyase deficiency",3,POMC,1
4,3,2-methyl-3-hydroxybutyric aciduria,4,HSD17B10,1
...,...,...,...,...,...
305065,6953,zoonoses,787,IFITM3,1
305066,6953,zoonoses,3361,IFIT2,1
305067,6954,zoonotic form of cutaneous leishmaniasis,104,CCL2,1
305068,6954,zoonotic form of cutaneous leishmaniasis,136,CXCL8,1


,document_no,document,vocabulary_no,vocabulary,count
0,0,(idiopathic) normal pressure hydrocephalus,0,APOE,1
1,0,(idiopathic) normal pressure hydrocephalus,1,AQP4,1
2,0,(idiopathic) normal pressure hydrocephalus,2,ASRGL1,1
3,0,(idiopathic) normal pressure hydrocephalus,3,BCAN,1
4,0,(idiopathic) normal pressure hydrocephalus,4,C9orf72,1
...,...,...,...,...,...
615868,6953,zoonoses,3537,AHI1,1
615869,6953,zoonoses,3744,NBEAL2,1
615870,6953,zoonoses,6584,SH2D3A,1
615871,6954,zoonotic form of cutaneous leishmaniasis,107,HSPA4,1


,document_no,document,vocabulary_no,vocabulary,count
0,0,(idiopathic) normal pressure hydrocephalus,0,CSF2,1
1,0,(idiopathic) normal pressure hydrocephalus,1,LAMC2,1
2,1,11-beta-hydroxylase deficiency,2,CYP11B1,1
3,1,11-beta-hydroxylase deficiency,3,CYP11B2,1
4,1,11-beta-hydroxylase deficiency,4,CYP21A2,1
...,...,...,...,...,...
237583,6952,zollinger-ellison syndrome,953,GAST,1
237584,6953,zoonoses,110,SAMD9,1
237585,6953,zoonoses,369,PRNP,1
237586,6953,zoonoses,1892,BST2,1


# Decide Topic Number by community detection

In [ ]:
element_name_list = [
    'DisGeNet_AlteredExpression', 
    'DisGeNet_Biomarker', 
    'DisGeNet_GeneticVariation', 
]
element_name_list = sorted(element_name_list)
element_name_list

In [ ]:
n_trial = 20
li_topics = []

for el in element_name_list:    
    data = df_comb.query('element_name == @el')

    # make Disease-Disease graph
    df_vs_element = data.explode('element').groupby('element', as_index=False).agg({
        'key_c': lambda x: x.tolist()
    })

    li_edgelist = []

    for ind, row in df_vs_element.iterrows():
        tmp = pd.DataFrame({
            'Description': row.element,
            'edges': [sorted(i) for i in itertools.combinations(row.key_c, 2)] #  B-A to A-B
        })
        li_edgelist.append(tmp)

    df_edgelist = pd.concat(li_edgelist, axis=0).assign(
        source = lambda df: df.edges.map(lambda x: x[0]),
        target = lambda df: df.edges.map(lambda x: x[1]),
        weight = 1
    ).drop('edges', axis=1).groupby(['source', 'target'], as_index=False).agg({
        'weight': sum
    })
    G = nx.from_pandas_edgelist(df_edgelist, "source", "target", ["weight"])

    for trial in range(n_trial):
        # clustering by the Louvain method
        com_G = community.best_partition(G)

        # store clustering result
        df_com_G = pd.DataFrame({'community': pd.Series(com_G)}).reset_index().rename(
            columns={'index': 'key_c'}
        ).assign(
            element_name = el,
            trial = trial
        )
        li_topics.append(df_com_G)

df_topics = pd.concat(li_topics)


In [ ]:
import plotly.express as px

df_topics_summary = df_topics.groupby(['element_name', 'trial'], as_index=False).agg({
    'community': lambda x: len(x.unique())
})

fig = px.histogram(df_topics_summary, x='community', color='element_name', marginal='box', nbins=200)
fig.update_traces(opacity=0.75)
fig.update_layout(barmode='stack')
fig.update_xaxes(range=[0, 12])
fig.show()

# Decided topic number is 6

In [6]:
# Run multimodal topic modeling in Gv(GeneticVariation)
N_topic = 6

N_extract = 290
rng = np.random.default_rng(10)

Disease_list = df_ae[['key_c']].assign(
    rand_value = lambda df: rng.permutation(df.shape[0])
).assign(
    Group = lambda df: df.rand_value // N_extract
)

Disease_list

for i, gdata in Disease_list.groupby('Group'):
    fname = f'result/Gv/predicted_w_group_{i}.csv'

    if os.path.exists(fname):
        continue
    
    print(f'Group {i}/{Disease_list.Group.max()} start')

    # make 290 diseases removed data in Gv 
    corpus_disease_gv = MyCorpus.read_corpus_from_dataframe(
        df_gv.drop(gdata.index, axis=0), 
        columns_title='key_c',
        columns_text='element'
    )

    # Run multimodal topic modeling ===================== 
    lda = MultiLDA(N_topic=N_topic)
    lda.load_disease({
        'AlteredExpression': corpus_disease_ae, 
        'Biomarker': corpus_disease_bm,
        'GeneticVatiation': corpus_disease_gv,
    })
    lda.fit(n_sample=5000, w_replace=False)
    w_esitmated = lda.get_w_estimated()
    # ===================================================
    
    w_esitmated.to_csv(fname)

Group 0/23 start
['AlteredExpression', 'Biomarker', 'GeneticVatiation']
 4950/5000Group 1/23 start
['AlteredExpression', 'Biomarker', 'GeneticVatiation']
 4950/5000Group 2/23 start
['AlteredExpression', 'Biomarker', 'GeneticVatiation']
 4950/5000Group 3/23 start
['AlteredExpression', 'Biomarker', 'GeneticVatiation']
 4950/5000Group 4/23 start
['AlteredExpression', 'Biomarker', 'GeneticVatiation']
 4950/5000Group 5/23 start
['AlteredExpression', 'Biomarker', 'GeneticVatiation']
 4950/5000Group 6/23 start
['AlteredExpression', 'Biomarker', 'GeneticVatiation']
 4950/5000Group 7/23 start
['AlteredExpression', 'Biomarker', 'GeneticVatiation']
 4950/5000Group 8/23 start
['AlteredExpression', 'Biomarker', 'GeneticVatiation']
 4950/5000Group 9/23 start
['AlteredExpression', 'Biomarker', 'GeneticVatiation']
 4950/5000Group 10/23 start
['AlteredExpression', 'Biomarker', 'GeneticVatiation']
 4950/5000Group 11/23 start
['AlteredExpression', 'Biomarker', 'GeneticVatiation']
 4950/5000Group 12/23 st